# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Reshape, concatenate, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4577502710435219443
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 9987122527097816325
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [6]:
len(english_sentences), len(french_sentences)

(137861, 137861)

From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    text_tokenizer = Tokenizer()
    text_tokenizer.fit_on_texts(x)
    text_tokenized = text_tokenizer.texts_to_sequences(x)
    
    return text_tokenized, text_tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    max_length = 0
    
    if length!=None:
        max_length = length
    else:
        for i in x:
            if len(i) > max_length:
                max_length = len(i)
                
    return pad_sequences(x, maxlen=max_length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [11]:
preproc_english_sentences.shape, preproc_french_sentences.shape

((137861, 15), (137861, 21, 1))

In [12]:
preproc_english_sentences[0]

array([17, 23,  1,  8, 67,  4, 39,  7,  3,  1, 55,  2, 44,  0,  0], dtype=int32)

In [13]:
preproc_french_sentences[0]

array([[ 35],
       [ 34],
       [  1],
       [  8],
       [ 67],
       [ 37],
       [ 11],
       [ 24],
       [  6],
       [  3],
       [  1],
       [112],
       [  2],
       [ 50],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0]], dtype=int32)

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [14]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [15]:
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
tmp_x.shape

(137861, 21, 1)

In [16]:
input_shape = tmp_x.shape

In [19]:
inputs = Input(shape=(input_shape[1:]))
RNN_1 = GRU(units=100, return_sequences=True)(inputs)
outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(RNN_1)
    
model = Model(inputs=inputs, outputs=outputs)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 21, 100)           30600     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 344)           34744     
Total params: 65,344
Trainable params: 65,344
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    inputs = Input(shape=(input_shape[1:]))
    RNN_1 = GRU(units=100, return_sequences=True)(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(RNN_1)
    
    model = Model(inputs=inputs, outputs=outputs)
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 80us/step - loss: 3.1298 - acc: 0.4298 - val_loss: nan - val_acc: 0.4676
Epoch 2/10
110288/110288 [==============================] - 7s 61us/step - loss: 2.2078 - acc: 0.5046 - val_loss: nan - val_acc: 0.5526
Epoch 3/10
110288/110288 [==============================] - 7s 61us/step - loss: 1.8477 - acc: 0.5654 - val_loss: nan - val_acc: 0.5751
Epoch 4/10
110288/110288 [==============================] - 7s 62us/step - loss: 1.6695 - acc: 0.5844 - val_loss: nan - val_acc: 0.5899
Epoch 5/10
110288/110288 [==============================] - 7s 62us/step - loss: 1.5696 - acc: 0.5981 - val_loss: nan - val_acc: 0.6040
Epoch 6/10
110288/110288 [==============================] - 7s 61us/step - loss: 1.4981 - acc: 0.6093 - val_loss: nan - val_acc: 0.6114
Epoch 7/10
110288/110288 [==============================] - 7s 61us/step - loss: 1.4377 - acc: 0.6170 - val_loss: nan - val_acc: 0.62

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [21]:
inputs = Input(shape=(input_shape[1],))
embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=100)(inputs)
RNN_1 = GRU(units=100, return_sequences=True)(embd_inputs)
outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(RNN_1)
    
model = Model(inputs=inputs, outputs=outputs)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 21, 100)           19900     
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 100)           60300     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 344)           34744     
Total params: 114,944
Trainable params: 114,944
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inputs = Input(shape=(input_shape[1],))
    embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=100)(inputs)
    RNN_1 = GRU(units=100, return_sequences=True)(embd_inputs)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(RNN_1)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 76us/step - loss: 3.5268 - acc: 0.4026 - val_loss: nan - val_acc: 0.4094
Epoch 2/10
110288/110288 [==============================] - 8s 73us/step - loss: 2.3617 - acc: 0.4970 - val_loss: nan - val_acc: 0.5824
Epoch 3/10
110288/110288 [==============================] - 8s 73us/step - loss: 1.5434 - acc: 0.6370 - val_loss: nan - val_acc: 0.6914
Epoch 4/10
110288/110288 [==============================] - 8s 73us/step - loss: 1.1175 - acc: 0.7270 - val_loss: nan - val_acc: 0.7577
Epoch 5/10
110288/110288 [==============================] - 8s 73us/step - loss: 0.8884 - acc: 0.7724 - val_loss: nan - val_acc: 0.7905
Epoch 6/10
110288/110288 [==============================] - 8s 73us/step - loss: 0.7452 - acc: 0.8011 - val_loss: nan - val_acc: 0.8154
Epoch 7/10
110288/110288 [==============================] - 8s 73us/step - loss: 0.6411 - acc: 0.8248 - val_loss: nan - val_acc: 0.83

In [23]:
tmp_x[:1].shape

(1, 21)

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [32]:
inputs = Input(shape=(input_shape[1:]))
inputs_reshape = Reshape((-1,))(inputs)
embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=100)(inputs_reshape)
Bi_RNN_1 = Bidirectional(GRU(units=100, return_sequences=True), merge_mode='concat')(embd_inputs)
Bi_RNN_2 = Bidirectional(GRU(units=100, return_sequences=True), merge_mode='concat')(Bi_RNN_1)
outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(Bi_RNN_2)
    
model = Model(inputs=inputs, outputs=outputs)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 21)                0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 21, 100)           19900     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 21, 200)           120600    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 21, 200)           180600    
_________________________________________________________________
time_distributed_16 (TimeDis (None, 21, 344)           69144     
Total params: 390,244
Trainable params: 390,244
Non-trainable params: 0
_________________________________________________________________
None

In [34]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inputs = Input(shape=(input_shape[1:]))
    inputs_reshape = Reshape((-1,))(inputs)
    embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=100)(inputs_reshape)
    Bi_RNN_1 = Bidirectional(GRU(units=100, return_sequences=True), merge_mode='concat')(embd_inputs)
    Bi_RNN_2 = Bidirectional(GRU(units=100, return_sequences=True), merge_mode='concat')(Bi_RNN_1)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(Bi_RNN_2)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_bd_model(bd_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 26s 234us/step - loss: 2.9350 - acc: 0.4581 - val_loss: nan - val_acc: 0.5046
Epoch 2/10
110288/110288 [==============================] - 24s 220us/step - loss: 1.7182 - acc: 0.5845 - val_loss: nan - val_acc: 0.6579
Epoch 3/10
110288/110288 [==============================] - 24s 221us/step - loss: 1.1256 - acc: 0.7091 - val_loss: nan - val_acc: 0.7509
Epoch 4/10
110288/110288 [==============================] - 24s 221us/step - loss: 0.8313 - acc: 0.7784 - val_loss: nan - val_acc: 0.8031
Epoch 5/10
110288/110288 [==============================] - 24s 221us/step - loss: 0.6541 - acc: 0.8218 - val_loss: nan - val_acc: 0.8381
Epoch 6/10
110288/110288 [==============================] - 24s 220us/step - loss: 0.5321 - acc: 0.8504 - val_loss: nan - val_acc: 0.8613
Epoch 7/10
110288/110288 [==============================] - 24s 220us/step - loss: 0.4526 - acc: 0.8701 - val_loss: nan -

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [50]:
# encoder RNN
inputs = Input(shape=(input_shape[1],))
embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=100)(inputs)
RNN_1 = GRU(units=100, return_sequences=False)(embd_inputs)
decode_inputs = RepeatVector(21)(RNN_1)

# decoder RNN
RNN_2 = GRU(units=100, return_sequences=True)(decode_inputs)
batch_RNN_2 = BatchNormalization(axis=-1)(RNN_2)
outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(batch_RNN_2)

model = Model(inputs=inputs, outputs=outputs)
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_32 (Embedding)     (None, 21, 100)           19900     
_________________________________________________________________
gru_56 (GRU)                 (None, 100)               60300     
_________________________________________________________________
repeat_vector_16 (RepeatVect (None, 21, 100)           0         
_________________________________________________________________
gru_57 (GRU)                 (None, 21, 100)           60300     
_________________________________________________________________
batch_normalization_6 (Batch (None, 21, 100)           400       
_________________________________________________________________
time_distributed_36 (TimeDis (None, 21, 344)           34744     
Total para

In [43]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    # encoder RNN
    inputs = Input(shape=(input_shape[1:]))
    inputs_reshape = Reshape((-1,))(inputs)
    embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=100)(inputs_reshape)
    RNN_1 = GRU(units=100, return_sequences=False)(embd_inputs)
    decode_inputs = RepeatVector(output_sequence_length)(RNN_1)

    # decoder RNN
    RNN_2 = GRU(units=100, return_sequences=True)(decode_inputs)
    batch_RNN_2 = BatchNormalization(axis=-1)(RNN_2)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(batch_RNN_2)

    model = Model(inputs=inputs, outputs=outputs)
    learning_rate = 1e-2
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_encdec_model(encdec_model)

# OPTIONAL: Train and Print prediction(s)
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 14s 128us/step - loss: 2.2367 - acc: 0.5055 - val_loss: nan - val_acc: 0.5095
Epoch 2/10
110288/110288 [==============================] - 12s 106us/step - loss: 1.1957 - acc: 0.6581 - val_loss: nan - val_acc: 0.6271
Epoch 3/10
110288/110288 [==============================] - 12s 106us/step - loss: 0.7132 - acc: 0.7795 - val_loss: nan - val_acc: 0.8086
Epoch 4/10
110288/110288 [==============================] - 12s 106us/step - loss: 0.4855 - acc: 0.8495 - val_loss: nan - val_acc: 0.8906
Epoch 5/10
110288/110288 [==============================] - 12s 106us/step - loss: 0.2784 - acc: 0.9166 - val_loss: nan - val_acc: 0.9209
Epoch 6/10
110288/110288 [==============================] - 12s 106us/step - loss: 0.2149 - acc: 0.9373 - val_loss: nan - val_acc: 0.9339
Epoch 7/10
110288/110288 [==============================] - 12s 106us/step - loss: 0.2303 - acc: 0.9352 - val_loss: nan -

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [17]:
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2]))
tmp_x.shape

(137861, 15)

In [18]:
from keras.callbacks import LearningRateScheduler
def lr_decay(epoch):
    lr_matrix = np.ones(18)
    lr_matrix[0:12] = 5e-3
    lr_matrix[12:16] = 2e-3
    lr_matrix[16:] = 1e-3
    #lr_matrix[17:] = 0.8e-3
    
    return lr_matrix[epoch]

lrrate = LearningRateScheduler(lr_decay)
callbacks_list = [lrrate]

In [19]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # encoder RNN
    inputs = Input(shape=(input_shape[1],))
    embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=200)(inputs)
    Bi_RNN_1 = Bidirectional(GRU(units=100, return_sequences=False), merge_mode='concat')(embd_inputs)
    decode_inputs = RepeatVector(output_sequence_length)(Bi_RNN_1)

    # decoder RNN
    Bi_RNN_2 = Bidirectional(GRU(units=100, return_sequences=True), merge_mode='concat')(decode_inputs)
    batch_RNN_2 = BatchNormalization(axis=-1)(Bi_RNN_2)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(batch_RNN_2)

    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 5e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_model_final(model_final)

print('Final Model Loaded')
# TODO: Train the final model
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2]))

# TODO: Train the neural network
final_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)
final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=18, validation_split=0.2, 
                    callbacks=callbacks_list, verbose=1)

# TODO: Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/18
110288/110288 [==============================] - 24s 214us/step - loss: 1.6661 - acc: 0.5943 - val_loss: 1.7913 - val_acc: 0.5652
Epoch 2/18
110288/110288 [==============================] - 23s 206us/step - loss: 0.5942 - acc: 0.8131 - val_loss: 0.7073 - val_acc: 0.7669
Epoch 3/18
110288/110288 [==============================] - 23s 206us/step - loss: 0.3074 - acc: 0.8988 - val_loss: 0.3057 - val_acc: 0.9017
Epoch 4/18
110288/110288 [==============================] - 23s 206us/step - loss: 0.2020 - acc: 0.9356 - val_loss: 0.1967 - val_acc: 0.9371
Epoch 5/18
110288/110288 [==============================] - 23s 206us/step - loss: 0.1623 - acc: 0.9488 - val_loss: 0.1649 - val_acc: 0.9478
Epoch 6/18
110288/110288 [==============================] - 23s 206us/step - loss: 0.1246 - acc: 0.9609 - val_loss: 0.1241 - val_acc: 0.9620
Epoch 7/18
110288/110288 [==============================] - 23s 206us/step - loss: 0

## Prediction (IMPLEMENTATION)

In [20]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = final_rnn_model

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 6: Custom (Enhancement)
The Model enhanced from the developed Model 5

Learning Rate Scheduler 

In [21]:
def lr_decay(epoch):
    lr_matrix = np.ones(22)
    lr_matrix[0:12] = 5e-3
    lr_matrix[12:15] = 2e-3
    lr_matrix[15:] = 1e-3
    #lr_matrix[17:] = 0.8e-3
    
    return lr_matrix[epoch]
    

In [22]:
from keras.callbacks import LearningRateScheduler

In [23]:
def model_final_enhance1(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # encoder RNN
    inputs = Input(shape=(input_shape[1],))
    embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=200)(inputs)
    Bi_RNN_1 = Bidirectional(GRU(units=200, return_sequences=False), merge_mode='concat')(embd_inputs)
    decode_inputs = RepeatVector(output_sequence_length)(Bi_RNN_1)

    # decoder RNN
    Bi_RNN_2 = Bidirectional(GRU(units=200, return_sequences=True), merge_mode='concat')(decode_inputs)
    batch_RNN_2 = BatchNormalization(axis=-1)(Bi_RNN_2)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(batch_RNN_2)

    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 5e-3
    
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


In [24]:
lrrate = LearningRateScheduler(lr_decay)
callbacks_list = [lrrate]

In [25]:
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)

# Instaintiate the Model
final_rnn_model_enhance1 = model_final_enhance1(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

# Train the Model
final_rnn_model_enhance1.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=22, validation_split=0.2, 
                             callbacks=callbacks_list, verbose=1)

print(logits_to_text(final_rnn_model_enhance1.predict(tmp_x[:1])[0], french_tokenizer))


Train on 110288 samples, validate on 27573 samples
Epoch 1/22
110288/110288 [==============================] - 37s 337us/step - loss: 1.4525 - acc: 0.6245 - val_loss: 1.6996 - val_acc: 0.5762
Epoch 2/22
110288/110288 [==============================] - 37s 334us/step - loss: 0.4518 - acc: 0.8516 - val_loss: 0.8486 - val_acc: 0.7311
Epoch 3/22
110288/110288 [==============================] - 37s 334us/step - loss: 0.2477 - acc: 0.9192 - val_loss: 0.2250 - val_acc: 0.9282
Epoch 4/22
110288/110288 [==============================] - 37s 334us/step - loss: 0.1514 - acc: 0.9522 - val_loss: 0.1352 - val_acc: 0.9576
Epoch 5/22
110288/110288 [==============================] - 37s 334us/step - loss: 0.1109 - acc: 0.9651 - val_loss: 0.1083 - val_acc: 0.9667
Epoch 6/22
110288/110288 [==============================] - 37s 334us/step - loss: 0.0925 - acc: 0.9711 - val_loss: 0.1082 - val_acc: 0.9673
Epoch 7/22
110288/110288 [==============================] - 37s 334us/step - loss: 0.0792 - acc: 0.9754

## Prediction - Model 6 (Enahnacement from Model 5)

In [27]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = final_rnn_model_enhance1

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 7: Custom (Model 5 without Batch-Norm)
The Model from the developed Model 5 without Batch-Normalization

In [32]:
def model_final_wo_batchn(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    # encoder RNN
    inputs = Input(shape=(input_shape[1],))
    embd_inputs = Embedding(input_dim=english_vocab_size, output_dim=200)(inputs)
    Bi_RNN_1 = Bidirectional(GRU(units=100, return_sequences=False), merge_mode='concat')(embd_inputs)
    decode_inputs = RepeatVector(output_sequence_length)(Bi_RNN_1)

    # decoder RNN
    Bi_RNN_2 = Bidirectional(GRU(units=100, return_sequences=True), merge_mode='concat')(decode_inputs)
    #batch_RNN_2 = BatchNormalization(axis=-1)(Bi_RNN_2)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(Bi_RNN_2)

    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 5e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


In [33]:
lrrate = LearningRateScheduler(lr_decay)
callbacks_list = [lrrate]

In [35]:
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)

# Instaintiate the Model
final_rnn_model_wo_batchn = model_final_wo_batchn(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Train the Model
final_rnn_model_wo_batchn.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=22, validation_split=0.2, 
                              callbacks=callbacks_list, verbose=1)

print(logits_to_text(final_rnn_model_wo_batchn.predict(tmp_x[:1])[0], french_tokenizer))


Train on 110288 samples, validate on 27573 samples
Epoch 1/22
110288/110288 [==============================] - 23s 205us/step - loss: 2.3450 - acc: 0.4853 - val_loss: nan - val_acc: 0.5855
Epoch 2/22
110288/110288 [==============================] - 22s 201us/step - loss: 1.3145 - acc: 0.6509 - val_loss: nan - val_acc: 0.7105
Epoch 3/22
110288/110288 [==============================] - 22s 201us/step - loss: 0.9400 - acc: 0.7359 - val_loss: nan - val_acc: 0.7581
Epoch 4/22
110288/110288 [==============================] - 22s 201us/step - loss: 0.7141 - acc: 0.7927 - val_loss: nan - val_acc: 0.8178
Epoch 5/22
110288/110288 [==============================] - 22s 201us/step - loss: 0.5091 - acc: 0.8535 - val_loss: nan - val_acc: 0.8866
Epoch 6/22
110288/110288 [==============================] - 22s 201us/step - loss: 0.3481 - acc: 0.9034 - val_loss: nan - val_acc: 0.9206
Epoch 7/22
110288/110288 [==============================] - 22s 201us/step - loss: 0.2613 - acc: 0.9285 - val_loss: nan -

## Prediction - Model 5 without Batch Normalization

In [36]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = final_rnn_model_wo_batchn

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [44]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 411916 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?